In [3]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm 
from datetime import datetime 
from keras import callbacks





In [13]:
w, h = 20, 20
base_path = os.path.join('/Users/25yoon/PycharmProjects/magister/dataset/gomocup2019results', '*.npz') # /Users/25yoon/PycharmProjects/magister/dataset/gomocup2019results

file_list = glob(base_path)

x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

# this are all the x data (the original point), and y data the response point 
x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))
print(x_data.shape)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


100%|██████████| 10224/10224 [00:06<00:00, 1611.96it/s]


(5643912, 20, 20, 1)
(4515129, 20, 20, 1) (4515129, 400)
(1128783, 20, 20, 1) (1128783, 400)


In [21]:

model = models.Sequential([
    layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(h, w, 1)),  # 3x3 filter
    layers.Conv2D(64, 3, activation='relu', padding='same'),  # Second convolutional layer
    layers.Conv2D(128, 3, activation='relu', padding='same'),  # A deeper layer to capture more complex patterns
    layers.Conv2D(1, 1, activation=None, padding='same'),  # 1x1 convolution to reduce output channels
    layers.Reshape((h * w,)),
    layers.Activation('sigmoid')  # Sigmoid activation for binary output
])


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 20, 20, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 20, 20, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 20, 20, 1)      │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 400)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,801 (362.50 KB)

 Trainable params: 92,801 (362.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from keras.src.callbacks import ModelCheckpoint, ReduceLROnPlateau

start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.keras' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    
    
)

Epoch 1/10
17638/17638 ━━━━━━━━━━━━━━━━━━━━ 0s 931ms/step - acc: 0.2470 - loss: 0.0147
Epoch 1: val_acc improved from -inf to 0.34920, saving model to ./models/20241022_170057.keras
17638/17638 ━━━━━━━━━━━━━━━━━━━━ 16867s 956ms/step - acc: 0.2470 - loss: 0.0147 - val_acc: 0.3492 - val_loss: 0.0091 - learning_rate: 0.0010
Epoch 2/10
10534/17638 ━━━━━━━━━━━━━━━━━━━━ 44:41 377ms/step - acc: 0.3503 - loss: 0.0090

In [17]:
print(tf.__version__)

2.17.0
